In [61]:
import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))
class GF():
    def __init__(self, f):
        Max = np.max(f)
        f2 = np.zeros(Max+1, np.uint8)
        f2[f] = 1
        f = np.flip(f2)
        self.len_of_symbol = Max
        self.q = 2**self.len_of_symbol
        self.table = np.array(self.make_code_table(f))
        
    def sub_matrix(self, A, i, j):
        A1 = np.delete(A, i, 0)
        A2 = np.delete(A1, j, 1)
        return A2

    def det(self, A):
        size = A.shape
        if size[0]==2 and size[1]==2:
            d = self.add(self.multi(A[0, 0],A[1, 1]), self.multi(A[1, 0],A[0, 1]))
            return d
        else:
            d = 0
            for i in range(size[0]):
                d=self.add(self.multi(A[0, i],self.det(self.sub_matrix(A, 0, i))), d)
            return d
        
    def make_code_table(self, f):
        m = len(f)-1
        N = 2**m
        table = ["0"*m]
        x =""
        for i in range(1,m+1):
            if f[i]==0:
                x+="0"
            else:
                x+="1"
        for i in range(0, m):
            c="0"*m
            c = c[0:i] + "1"+c[i+1:]
            c = c[::-1]
            table.append(c)
        for i in range(m+1, N):
            a = table[-1]
            a1 = a[1:]+"0"
            b = "0"*m
            if a[0]=="1":
                b = x
            c = self.add_string_code(a1, b)
            table.append(c)
        return table

    def add_string_code(self, a, b):
        c =""
        for i in range(self.len_of_symbol):
            if a[i]!= b[i] :
                c+="1"
            else:
                c+="0"
        return c
    def add(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
            a= int(a)
            b = int(b)
            a1 = self.table[a]
            b1 = self.table[b]
            c1 = self.add_string_code(a1, b1)
            c, = np.where(self.table == c1)
            return c[0]
        elif not np.isscalar(a):
            if np.isscalar(b):
                raise ValueError("a, b must have same shape ( a:",a.shape,", b is scalar)" )
            else:
                a_size = a.shape
                b_size = b.shape
                if  len(a_size) ==2 and  len(a_size)==2:
                    if a_size[0] != b_size[0] or a_size[1] != b_size[1] :
                        raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )
                    else:
                        c = np.zeros_like(a, np.uint8)
                        for i in range(a_size[0]):
                            for j in range(a_size[1]):
                                c[i, j] = self.add(a[i, j], b[i, j])
                        return c
                elif len(a_size)==1 and len(b_size)==1 and a_size ==b_size :
                    c = np.zeros_like(a, np.uint8)
                    for i in range(a_size[0]):
                        c[i] = self.add(a[i], b[i])
                    return c
                else:
                    raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )         
    def multi(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
                if a ==0 or b == 0:
                    return 0
                else:
                    c = (a+b-2) % (self.q-1)
                    return c+1
        elif not np.isscalar(a):
            a_size = a.shape
            if np.isscalar(b):
                c = [self.multi(i, b) for i in a]
                return np.array(c)
            b_size = b.shape
            if a_size[1] != b_size[0] :
                raise ValueError("Can not multiply 2 matrix of dimension: ", a.shape, b.shape)
            else:
                c = np.zeros((a_size[0], b_size[1]), np.uint8)
                for i in range(a_size[0]):
                    for j in range(b_size[1]):
                        for k in range(a_size[1]):
                            c[i, j] =self.add(c[i, j], self.multi(a[i, k], b[k, j])) 
                return c
    def make_sub_matrix(self, x):
        N =  N = self.q -1
        sub = np.zeros((N, N), np.uint8)
        index = x-1
        for i in range(N):
            sub[i, index] = index+1
            index +=1
            if index ==N:
                index =0
        return sub
    def reserve_e(self, a):
            a2 = self.q+1-a
            if a2 == self.q:
                a2 = 1
            if a2 == self.q+1:
                a2 = 0
            return a2
    def Reserve(self, X, show =False):
        N = X.shape[0]
        def make_I(n):
                In = np.zeros((n, n), np.uint8)
                for i in range(n):
                    In[i, i] =1
                return In
        X1 = make_I(N)
        for i in range(N):
            if show:
                print(i)
            C = make_I(N)
            for j in range(N):
                C[j, i] = self.multi(X[j, i], self.reserve_e(X[i,i]))
            C[i,i] = self.reserve_e(X[i,i])
            X = self.multi(C,X)
            X1 = self.multi(C,X1)
        return X1
    
import numpy as np
from scipy.special import erfc

def S2B(symbol, table):
    b = table[symbol]
    B = np.array(list(b)).astype(np.float32)
    return B

def probB(x, N0):
    d1 = abs(-1-x)
    d2 = abs(1-x)
    p0 = np.e**(-d1/N0)/(np.e**(-d1/N0)+np.e**(-d2/N0))
    p1 = 1-p0
    return [p0, p1]

def probS(S, N0, q, table, M):
    B = np.zeros((q, 2))
    for i in range(q):
        B[i] = probB(S[i], N0)
    P = np.zeros(M)
    for i in range(M):
        Bi = S2B(i, table)
        p = 1
        for j in range(q):
            p *=B[j, int(Bi[j])]
        P[i] = p
    m = np.max(P)
    P = np.log10(m/P)
    return P

def BPSK(message, M, table, SNRdB):
    Eb = 1
    N0 = Eb/10**(SNRdB/10)
    q = int(np.log2(M))
    Ns = len(message)
    Nb = Ns*q
    noise = np.sqrt(N0/2)*np.random.randn(Nb)
    noise = noise.reshape((Ns, q))
    X = np.zeros_like(noise)
    for i in range(Ns):
        X[i]= S2B(message[i], table)
    X[X==0]=-1
    Y = X+noise
    output = np.zeros((Ns, M))
    for i in range(Ns):
        output[i] = probS(Y[i], N0, q, table, M)
    return output

def HardDicision(Message):
    return np.argmin(Message, axis=1)

def score(X, Y):
    Z = np.zeros_like(X)
    Z[X!=Y]=1
    return np.sum(Z)

def predict(Q):
    message = np.argmin(Q, axis=0)
    X = 0
    for i in range(dc):
        X = gl.add(X, message[i])
    return message, X

def permutation(X, hi):
    Y = np.zeros_like(X)
    Y[0] = X[0]
    for i in range(1, X.shape[0]):
        Y[gl.multi(i, hi)] = X[i]
    return Y

def permutation2d(X, Y):
    Z = np.zeros_like(X)
    for i in range(X.shape[1]):
        Z[:, i] = permutation(X[:, i], Y[i])
    return Z

def unpermutation2d(X, Y):
    Z = np.zeros_like(X)
    for i in range(X.shape[1]):
        Z[:, i] = permutation(X[:, i], gl.reserve_e(Y[i]))
    return Z

def convert_vhdl(A, elem32=False):
    shape = A.shape
    s = "("
    if len(shape)==1:
        for i in A:
            si = int(i)
            s+=str(si)+", "
        s=s[:-2]+")"
        return s
    if len(shape)==2:
        for row in A:
            print(convert_vhdl(row)+",")

def debug(name, value, debug=False):
    if debug:
        print(name+"#"*30)
        print(value)
        
def row_selection(Q, H_i, H_non_zeros_index_i):
    Qi = np.zeros((q, dc))
    Qi = Q[:,H_non_zeros_index_i]
    Qi = permutation2d(Qi, H_i[H_non_zeros_index_i])
    return Qi

def row_update(Q, Qi2, H_i, H_non_zeros_index_i):
    Qi2 = unpermutation2d(Qi2,H_i[H_non_zeros_index_i])
    Q[:, H_non_zeros_index_i] = Qi2
    return Q
    
def CN(Q, dg=False):
    A = 0.5
    zn = np.argmin(Q, axis=0)
    debug("Zn", zn, dg)
    dQ = np.zeros_like(Q)
    for j in range(dc):
        for a in range(q):
            nj = gl.add(a, zn[j])
            dQ[nj, j] = Q[a, j]
            
    debug("dQ", dQ, dg)
    B = 0
    for i in range(dc):
        B = gl.add(B, zn[i])
    debug("B", B, dg)
    def sai(A):
        col = np.argmin(A)
        sortA = np.sort(A)
        return sortA[0], sortA[1], col

    m = np.zeros((q-1, 3))
    for i in range(1, q):
        m[i-1]= sai(dQ[i])
    debug("m", m, dg)
    debug("m0", m[:, 0], dg)
#     print(convert_vhdl(m[:, 0]))        
#     print("m1"+"#"*30)
#     print(convert_vhdl(m[:, 1], True))
    conf = np.ones((q, q))*1000000
    conf[0, 0] = 0
    conf[1:, 0] = m[:, 0]
    for i in range(1,q-1):
        for j in range(i+1, q):
            if m[i-1, 2]!=m[j-1,2]:
                conf[gl.add(i,j), i] = max(m[i-1, 0],m[j-1,0])
    debug("conf", conf, dg)
    extra_columm = np.zeros((q, 6))
    extra_columm[:, 0] = np.min(conf, axis=1)
    extra_columm[1:, 1] = m[:, 0]
    extra_columm[1:, 2] = m[:, 1]
    extra_columm[1:, 5] = m[:, 2]
    argmin=  np.argmin(conf, axis=1)
    for i in range(1, q):
        if argmin[i]==0:
            extra_columm[i, 3] = extra_columm[i, 4] = m[i-1, 2]
        else:
            d1 = m[argmin[i]-1, 2]
            d2 = m[gl.add(argmin[i], i)-1, 2]
            extra_columm[i, 3]=d1
            extra_columm[i, 4] =d2
    debug("extra_columm", extra_columm, dg)
    dR = np.zeros_like(dQ)
    
    def dRij(extra_colummi, j):
        dQi, m1, m2, d1, d2, mcol = extra_colummi
        if d1==j or d2==j:
            if d1 == d2:
                return m2
            else:
                if mcol ==j:
                    return m2
                else:
                    return m1
        else:
            return dQi
        
    for i in range(q):
        for j in range(dc):
            dR[i, j] = dRij(extra_columm[i], j)
    debug("dR", dR, dg)
    R = np.zeros_like(Q)
    for i in range(q):
        for j in range(dc):
            index = gl.add(i, B)
            index = gl.add(index, zn[j])
            R[index, j]= (A*dR[i,j])
            
    debug("R", R,dg) 
    Q = Q+R
    debug("Q", Q, dg)
    return R

def quick_check(H, C):
    C = C.reshape((C.shape[0], 1))
    return gl.multi(np.array(H), C)

gl = GF([3,  1, 0])
print(gl.table)

['000' '001' '010' '100' '011' '110' '111' '101']


In [132]:
m = 4
n = 8
dc = 3
q = gl.q
H= np.zeros((m, n), np.uint8)
indexs = np.zeros((m, dc), np.uint8)
for i in range(m):
    indexs[i] = np.random.permutation(n)[:dc]

values = np.random.randint(1, q, (m, dc))
for i in range(m):
    H[i][indexs[i]]=values[i]

print(H)

[[0 6 0 0 0 2 0 7]
 [0 7 3 0 0 0 4 0]
 [5 0 0 7 6 0 0 0]
 [0 0 1 0 5 0 1 0]]


In [133]:

c = None
s = 1
N=10
C = np.zeros((N, n), np.uint8)
stt = 0

# H = np.array([
#     [1, 0, 0, 4, 0, 3],
#     [0, 2, 0, 1, 6, 0],
#     [0, 0, 5, 0, 2, 1]
# ])


while True:
    c = np.random.randint(0, q, n).reshape((n, 1))
    
    s = np.sum(gl.multi(H, c))
    if s==0:
        C[stt] = c.reshape(n).copy()
        print(c.reshape(n))
        stt+=1
        if stt == N:
            break
    
    
H_non_zeros_index  = np.zeros((m, dc), np.uint32)
for i in range(m):
    stt = 0
    for j in range(n):
        if H[i, j]!=0:
            H_non_zeros_index[i, stt] = j
            stt+=1

[0 6 4 2 3 7 5 3]
[7 1 6 7 5 6 7 3]
[6 1 1 2 2 3 5 4]
[7 2 3 5 0 2 3 2]
[6 4 7 2 2 7 2 5]
[1 0 7 3 5 5 6 7]
[4 6 5 4 2 4 1 1]
[1 6 3 1 6 7 0 3]
[7 1 2 5 0 1 2 1]
[6 5 3 2 2 3 4 7]


In [81]:
c = np.array([0, 3, 1, 2, 1, 1])
Q0 = BPSK(c, q, gl.table, 3)
Q0 = Q0.transpose()
print(np.round(Q0, 5))

[[0.      1.73306 1.27903 1.73306 1.73306 1.00001]
 [1.73306 2.99925 0.24441 2.68326 0.      0.     ]
 [1.73306 1.88222 2.91056 0.      3.46613 2.67074]
 [1.73306 0.      1.03462 3.07283 3.46613 2.05342]
 [3.46613 3.1484  1.87593 0.9502  1.73306 1.67073]
 [3.46613 0.14915 2.66615 1.33977 5.19919 3.72415]
 [5.19919 1.41534 1.63153 2.28997 3.46613 2.72414]
 [3.46613 1.26618 0.      4.02303 1.73306 1.05342]]


In [92]:
for c in C:
    syn= quick_check(H, c)
    print(syn)

[[0]
 [0]
 [0]]
[[0]
 [0]
 [0]]
[[0]
 [0]
 [0]]
[[0]
 [0]
 [0]]
[[0]
 [0]
 [0]]
[[0]
 [0]
 [0]]
[[0]
 [0]
 [0]]
[[0]
 [0]
 [0]]
[[0]
 [0]
 [0]]
[[0]
 [0]
 [0]]


In [134]:
def decoder2(Q0):
    Q = Q0.copy()
    Rmna = np.zeros((m, q, dc))
    Qmna = np.zeros((m, q, dc))
    for i in range(m):
        Qmna[i] = permutation2d(Q[:, H_non_zeros_index[i]], H[i][H_non_zeros_index[i]])
    for t in range(5):
        stt = 0
        for i in range(m):
            Qmnai = permutation2d(Q[:, H_non_zeros_index[i]], H[i][H_non_zeros_index[i]])-Rmna[i]
            minQ = np.min(Qmnai, axis=0)
            Qmnai = Qmnai - minQ
            Rmna[i] = CN(Qmnai)
            QRmni = Rmna[i]+ Qmna[i]
            Qmna[i] = QRmni.copy()
            Q[:, H_non_zeros_index[i]] = unpermutation2d(QRmni,  H[i][H_non_zeros_index[i]])
        y, s = predict(Q)
        s = np.sum(quick_check(H, y))
        if s == 0:
            break
    return y

def decoder(Q0):
    y = None
    Q = Q0.copy()
    Rmna = np.zeros((m, q, n), np.int32)
    Qmna = np.zeros((m, q, n), np.int32)

    for i in range(m):
        Qmna[i] = Q.copy()

    for t in range(15):

        for i in range(m):
            Qmnai = permutation2d(Qmna[i][:, H_non_zeros_index[i]], H[i][H_non_zeros_index[i]])
            Rmnai = CN(Qmnai)
            Rmna[i][:, H_non_zeros_index[i]] = unpermutation2d(Rmnai, H[i][H_non_zeros_index[i]])

        R = np.zeros_like(Rmna[i])
        for i in range(m):
            R+=Rmna[i]
        for i in range(m):
            Qmna[i]  = Q+R- Rmna[i]
            Qmna[i] = Qmna[i] - np.min(Qmna[i], axis=0)
        Q = Q+R
        y, s = predict(Q)
        s = np.sum(quick_check(H, y))
        if s == 0:
            break
    return y

def test(N, SNRdB):
    NN = 0
    EE = 0
    E0 = 0
    EN = 0
    EN0 = 0 
    for i in range(N):
        rand = np.random.randint(0, C.shape[0])
        c0 = C[rand]
        c = c0.reshape(n)
        Q0 = BPSK(c, q, gl.table, SNRdB)
        Q0 = Q0.transpose()
        _, s = predict(Q0)
        e0 = score(c, _)
        E0+=e0
        if e0!=0:
            EN0+=1
        y = decoder2(Q0)
        e = score(c, y)
        EE+=e
        NN+=n
        if e!=0:
            EN+=1
        
    return NN, E0, EE, EN0, EN
    
NN, E0,  EE, EN0, EN = test(10000, 4)
print(NN, E0, EN0, EE, EN)

80000 2993 2631 69 31
